# 감성분석 모델 구축

In [ ]:
# ! pip install scikit-learn
# ! pip install konlpy

In [2]:
import re
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
# from konlpy.tag import Mecab
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences

warnings.filterwarnings(action = 'ignore') # warning 메시지 표시x

### 3. 훈련모델Ⅲ - 214,060개 (7:3)
- 네이버+쿠팡 리뷰합친 데이터에서,
- 7:3비율로 나눠서 훈련 -> 평가 진행

In [6]:
model3_naver_coupang = pd.read_excel('(4)all_reviews_214060.xlsx') # 214,060개

# 0. 훈련 데이터와 테스트 데이터를 3:1 비율로 분리
train_data, test_data = train_test_split(model3_naver_coupang, test_size = 0.25, random_state = 42)

print('훈련용 리뷰의 개수 :', len(train_data))
print('테스트용 리뷰의 개수 :', len(test_data))
# 훈련용 리뷰의 개수 : 160545
# 테스트용 리뷰의 개수 : 53515


# 1. 라벨 분포 확인
train_data['label'].value_counts()
# 1    85144
# 0    75401

test_data['label'].value_counts()
# 1    28308
# 0    25207

# 2. 한글 외의 문자 제거
train_data['reviews'] = train_data['reviews'].apply(lambda x : re.sub(r'[^ ㄱ-ㅣ가-힣]+', " ", x))
test_data['reviews'] = test_data['reviews'].apply(lambda x : re.sub(r'[^ ㄱ-ㅣ가-힣]+', " ", x))

# train_data.head()
# test_data.head()

훈련용 리뷰의 개수 : 160545
테스트용 리뷰의 개수 : 53515


In [7]:
# 3. 분석 모델 구축 전 작업

# 1)형태소 분석 - 문장을 토큰화
okt = Okt()
def okt_tokenizer(text):
    tokens = okt.morphs(text)
    return tokens

# 2)TF-IDF 벡터화에 사용할 tfidf 객체 생성 -> 벡터로 변환(transform)
tfidf = TfidfVectorizer(tokenizer=okt_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf.fit(train_data['reviews'])
model3_train_tfidf = tfidf.transform(train_data['reviews'])
model3_train_tfidf

# 18분 14초

<160545x160305 sparse matrix of type '<class 'numpy.float64'>'
	with 4127203 stored elements in Compressed Sparse Row format>

In [8]:
# 4. 분석 모델 구축

SA_lr = LogisticRegression(random_state=0)

SA_lr.fit(model3_train_tfidf, train_data['label'])

params = {'C': [1,3,3.5,4,4.5,5]}
SA_lr_grid_cv = GridSearchCV(SA_lr, param_grid=params, cv=3, scoring='accuracy', verbose=1)

SA_lr_grid_cv.fit(model3_train_tfidf, train_data['label'])

print(SA_lr_grid_cv.best_params_, round(SA_lr_grid_cv.best_score_, 4))

SA_lr_best1 = SA_lr_grid_cv.best_estimator_ 

# {'C': 3.5} 0.9162

Fitting 3 folds for each of 6 candidates, totalling 18 fits
{'C': 3.5} 0.9162


In [9]:
# 5. 분석 모델 평가 - 모델 정확도 확인

model3_test_tfidf = tfidf.transform(test_data['reviews'])

test_predict = SA_lr_best1.predict(model3_test_tfidf)

print('감성 분석 정확도 : ', round(accuracy_score(test_data['label'], test_predict), 3))
# 감성 분류 모델의 정확도가 91.8%!
# (3분 14초)

감성 분석 정확도 :  0.918


### Why? 
- 흠..model1에 비해 정확도가 떨어질 것이라 생각했는데, 오히려 오름
- 쨋든.. 이걸로 퍼센트 예측까지 하면 되려나..

In [ ]:
# model1은 네이버 쇼핑리뷰데이터만 활용해서, 훈련:테스트 비율을 3:1로 나눠서 진행했어. 그리고, 이건 감성분류 모델 정확도가 91.6퍼가 나왔어
# 그리고 model2는 네이버쇼핑리뷰와 쿠팡리뷰를 다 섞었고, 그 상태에서 3:1로 훈련:테스트 비율을 나눠서 진행했어.
    #근데 이건 감성 분류 모델 정확도가 오히려 높게 나왔어 91.8퍼.
# 내 생각에는 model1이 훨씬 높게 나올거라고 생각했어. 왜냐하면, 네이버 쇼핑리뷰와 쿠팡리뷰는 다루는 상품군의 범주가 많이 다르기 때문에 섞으면,
    # 감성 분석이 덜 진행될거라고 생각했단 말이야. 왜 이런 결과가 나오는 걸까?

# ----
# 1.다양성의 효과: 두 데이터 세트를 섞었을 때, 데이터의 다양성이 증가하여 모델이 더 넓은 범위의 언어 패턴을 학습할 수 있게 되었을 수 있습니다. 즉, 더 다양한 상품 리뷰를 포함함으로써 모델이 다양한 감정 표현을 보다 잘 이해하고 일반화할 수 있게 된 것일 수 있습니다.

# 2.훈련 데이터의 균형: 쿠팡 리뷰 데이터에는 블루투스 무선 이어폰 리뷰만 포함되어 있으므로, 이 특정 제품 범주에 대한 감성 표현이 더 풍부할 수 있습니다. 이는 특정 유형의 상품에 대한 감성을 모델이 더 정확하게 파악할 수 있게 도와줄 수 있습니다.

# 3.모델의 오버피팅 감소: 네이버 쇼핑 리뷰만 사용했을 때 모델이 해당 데이터의 특정 패턴에 과적합될 가능성이 있습니다. 반면, 다양한 데이터 소스를 섞음으로써 모델이 더 일반적인 언어 패턴을 학습하여 실제로 테스트 데이터에 대한 성능이 개선될 수 있습니다.

# 4.평가 기준의 차이: 각 데이터 세트에서 사용된 평가 기준이나 리뷰의 양상이 다를 수 있습니다. 예를 들어, 쿠팡의 리뷰가 네이버 쇼핑 리뷰보다 더 명확하게 긍정적 또는 부정적 감정을 표현할 수 있고, 이것이 모델이 감정을 분류하는 데 도움을 줄 수 있습니다.

# -> 이러한 결과를 확인하기 위해서는 더 많은 실험과 분석이 필요할 것입니다.
    # 예를 들어, 두 데이터 세트에서의 리뷰들의 언어적 특성을 분석하거나,
    # 더 다양한 모델 구성을 테스트하여 어떤 요소가 성능 향상에 기여하는지 파악해볼 수 있습니다.
    # 또한, 모델의 성능을 더 자세히 이해하기 위해 혼동 행렬(confusion matrix)을 사용하여 각 클래스(긍정적, 부정적)별 정확도를 확인해 보는 것도 좋은 방법입니다.

# 흠.. 그렇다면, 블루투스 이어폰 말고도 다른 상품군의 쿠팡 리뷰를 긁어서 여기에 합쳐서 하면 또 정확도가 더 오를것 같기도?
    # *이점을 향후 개선점으로 넣어도 좋을듯 하다
